In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], with_info=True, as_supervised=True)

In [ ]:
num_classes = metadata.features['label'].num_classes
num_classes

In [ ]:
get_label_name = metadata.features['label'].int2str

In [ ]:
image, label = next(iter(train_ds))

In [ ]:
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

In [ ]:
image.shape

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
result = resize_and_rescale(image)

In [ ]:
_ = plt.imshow(result)

In [ ]:
print("Min and Max of image: {}, {}".format(np.min(result), np.max(result)))

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip(),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
image = tf.expand_dims(image, 0)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    augmented_image = data_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])

In [ ]:
model = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
])

In [ ]:
aug_ds = train_ds.map(lambda x, y: (resize_and_rescale(x, training=True), y))

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)
    
    ds = ds.batch(batch_size)

    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
    
    return ds.prefetch(AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, True, True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(True), metrics=['accuracy'])

epochs = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
loss, acc = model.evaluate(test_ds)
acc

In [ ]:
def random_invert_img(x, p = 0.5):
    if tf.random.uniform([]) < 0.5:
        x = (255 - x)
    else:
        x
    return x

In [ ]:
def random_invert(factor=0.5):
    return tf.keras.layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
    augmented_image = random_invert(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0].numpy().astype('uint8'))

In [ ]:
class RandomInvert(tf.keras.layers.Layer):
    def __init__(self, factor=0.5, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor
    
    def call(self, x):
        return random_invert_img(x)

In [ ]:
_ = plt.imshow(RandomInvert()(image)[0])

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], with_info=True, as_supervised=True)

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

In [ ]:
def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1, 2, 1)
    plt.title('Origin image')
    plt.imshow(original)

    plt.subplot(1, 2, 2)
    plt.title('Augmented')
    plt.imshow(augmented)

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

In [ ]:
brighted = tf.image.adjust_brightness(image, 0.4)
visualize(image, brighted)

In [ ]:
croped = tf.image.central_crop(image, 0.5)
visualize(image, croped)

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

In [ ]:
for i in range(3):
    seed = (i, 0)
    stateless_random_brightness = tf.image.stateless_random_brightness(image, 0.95, seed)
    visualize(image, stateless_random_brightness)

In [ ]:
for i in range(3):
    seed = (i, 0)
    stateless_random_contrast = tf.image.stateless_random_contrast(image, 0.1, 0.9, seed)
    visualize(image, stateless_random_contrast)

In [ ]:
for i in range(3):
    seed = (i, 0)
    stateless_random_crop = tf.image.stateless_random_crop(image, [210, 300, 3], seed)
    visualize(image, stateless_random_crop)

In [ ]:
(train_datasets, val_ds, test_ds), metadata = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], with_info=True, as_supervised=True)

In [ ]:
def resize_and_rescale(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = (image / 255.0)
    return image, label

In [ ]:
def augment(image_label, seed):
    image, label = image_label
    image, label = resize_and_rescale(image, label)
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)

    new_seed = tf.random.experimental.stateless_split(seed, 1)[0, :]

    image = tf.image.stateless_random_crop(image, [IMG_SIZE, IMG_SIZE, 3], seed)

    image = tf.image.stateless_random_brightness(image, 0.5, new_seed)

    image = tf.clip_by_value(image, 0, 1)

    return image, label

In [ ]:
counter = tf.data.experimental.Counter()
train_ds = tf.data.Dataset.zip((train_datasets, (counter, counter)))

In [ ]:
train_ds = train_ds.shuffle(1000).map(augment, num_parallel_calls = AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
val_ds = val_ds.map(resize_and_rescale, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
test_ds = test_ds.map(resize_and_rescale, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
rng = tf.random.Generator.from_seed(123, alg='philox')

In [ ]:
def f(x, y):
    seed = rng.make_seeds(2)[0]
    image, label = augment((x, y), seed)
    return image, label

In [ ]:
train_ds = train_datasets.shuffle(1000).map(f, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
val_ds = val_ds.map(resize_and_rescale, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
test_ds = test_ds.map(resize_and_rescale, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)